 ### **SUPERAI2-SCAMPER5**.

In [ ]:
#!pip install typing-extensions
TrainSize = 250000 #10000000

In [ ]:
from ast import Str
import os
import numpy as np
from typing_extensions import TypedDict
from typing import Generator, List, NamedTuple, Iterable
import json
import cv2
import pandas as pd
from PIL import Image
from IPython.display import display

IMG_SIZE = 28


In [ ]:
FontMap = [
    "Angsana_New",
    "Cordia_New",
    "DM_Shining_Star_Regular",
    "FC_Knomphing_Regular",
    "fonttintin",
    "Kunlasatri",
    "TH_Chakra_Petch",
    "TH_Charm_of_AU",
    "TH_Mali_Grade6",
    "TH_Sarabun"]

FontFile = ["ANGSA.ttf", "CORDIA.ttf", "DM Shining Star Regular.ttf", "FC Knomphing Regular.ttf", "fonttintin.ttf", "Kunlasatri.ttf",
        "TH Chakra Petch.ttf", "TH Charm of AU.ttf", "TH Mali Grade6.ttf", "THSarabun.ttf"]

In [ ]:
class DataAndLabel(NamedTuple):
    Id : str
    PageW : int 
    Mat : np.ndarray
    Text : str
    FontFamily : int
    FontSize : int
    IsBold : bool
    IsItalic : bool
def readSetWithLabel(setpath = "train/set1") -> Generator[DataAndLabel, None, None]:
    class RectFormat(TypedDict):
        x: float
        y: float
        width: float
        height: float
    class StyleFormat(TypedDict):
        fontSize: str
        fontStyle: str
        fontWeight: str
        fontVarient: str
    class LoadFormat(TypedDict):
        text : str
        rect : RectFormat
        style : StyleFormat
    for mainpath in os.listdir(setpath):
        fname = f"{setpath}/{mainpath}/image.png"
        mat = cv2.imread(fname)
        fname = fname.replace("/", "_")
        with open(f"{setpath}/{mainpath}/gt_text.json", encoding="utf-8") as f:
            jSArray : List[LoadFormat] = json.load(f)
        for idx,JSON in enumerate(jSArray):
            text = JSON["text"]
            #Text = text
            rect = JSON["rect"]
            x = rect["x"]
            y = rect["y"]
            width = rect["width"]
            height = rect["height"]
            Mat = mat[round(y):round(y+height),round(x):round(x+width),:]
            style = JSON["style"]
            fontFamily =FontMap.index(style["fontFamily"])
            FontSize = int(style["fontSize"][:-2])
            IsItalic = style["fontStyle"] == "italic"
            IsBold = style["fontWeight"] == "bold"
            fid = fname + str(idx)
            yield DataAndLabel(fid,mat.shape[1], Mat, text, fontFamily, FontSize, IsBold, IsItalic)
        del mat
def readSetWithoutLabel(setpath = "train/set2") -> Generator[np.ndarray, None, None]:
    class RectFormat(TypedDict):
        x: float
        y: float
        width: float
        height: float
    class LoadFormat(TypedDict):
        id : str
        rect : RectFormat
    for mainpath in os.listdir(setpath):
        mat = cv2.imread(f"{setpath}/{mainpath}/image.png")
        with open(f"{setpath}/{mainpath}/gt_text.json", encoding="utf-8") as f:
            jSArray : List[LoadFormat] = json.load(f)
        for JSON in jSArray:
            rect = JSON["rect"]
            x = rect["x"]
            y = rect["y"]
            width = rect["width"]
            height = rect["height"]
            Mat = mat[round(y):round(y+height),round(x):round(x+width),:]
            yield Mat
        del mat
class DataTest(NamedTuple):
    Mat : np.ndarray
    Id : str
def readTestSet(setpath = "test") -> Generator[DataTest, None, None]:
    class RectFormat(TypedDict):
        x: float
        y: float
        width: float
        height: float
    class LoadFormat(TypedDict):
        id : str
        x : float
        y : float
        width : float
        height : float
    for mainpath in os.listdir(setpath):
        mat = cv2.imread(f"{setpath}/{mainpath}/image.png")
        with open(f"{setpath}/{mainpath}/test.json", encoding="utf-8") as f:
            jSArray : List[LoadFormat] = json.load(f)
        for JSON in jSArray:
            text = JSON["id"]
            x = JSON["x"]
            y = JSON["y"]
            width = JSON["width"]
            height = JSON["height"]
            Mat = mat[round(y):round(y+height),round(x):round(x+width),:]
            yield DataTest(Mat, text)
        del mat

In [ ]:
datapath = '../input/super-ai-engineer-2021-font-recognition/'

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
matplotlib.__version__  

In [ ]:
def preprocessImage(img_arr, CropSize=32, maximg=5, blockSize=5, C=2):
    imgList = []
    ImgW = img_arr.shape[1]
    ImgH = img_arr.shape[0]

    image_gray = 255 - cv2.cvtColor(img_arr, cv2.COLOR_BGR2GRAY)
    image_gray = cv2.equalizeHist(image_gray)

    if(ImgW < CropSize):
        image_gray = np.hstack((image_gray, np.zeros((ImgH, CropSize - ImgW))))
        ImgW = CropSize
    if(ImgH < CropSize):
        image_gray = np.vstack((image_gray, np.zeros((CropSize - ImgH, ImgW))))
        ImgH = CropSize

    sliceW = round(CropSize*2/3)
    minSum = CropSize*CropSize*255//10
    Skip = sliceW*3//5
    cnt = 0
    y = 0
    while y < ImgH - Skip:
        if y > ImgH - CropSize:
            y = ImgH - CropSize

        x = 0
        while x < ImgW - Skip:
            if x > ImgW - CropSize:
                x = ImgW - CropSize
            crop = image_gray[y:y + CropSize, x : x + CropSize]
            if np.sum(crop)> minSum:
                imgList.append(crop)
                cnt += 1
                if cnt >= maximg:
                    return imgList
            x += sliceW
        y += sliceW

    if(cnt == 0):
        crop = image_gray[:CropSize,:CropSize]
        imgList.append(crop)            

    return imgList


In [ ]:
!pip3 install pickle5

In [ ]:
import pickle5 as pickle
with open("../input/traincmix/dfTrainCMix.pkl", "rb") as fh:
  data = pickle.load(fh)
ImgAll = np.load('../input/traincmix/TrainCMix.npy')

In [ ]:
data

In [ ]:
df = data[:TrainSize]

In [ ]:
train, validate, test = np.split(df, [int(.7*len(df)), int(.85*len(df))])
X_train, X_val, X_test = np.split(ImgAll[:TrainSize], [int(.7*len(df)), int(.85*len(df))])

In [ ]:
from keras.utils.np_utils import to_categorical

def dfToY(df):
    yFam = to_categorical(df["FontFmaily"], num_classes = 10)
    yS   = to_categorical((df["FontSize"]-12)//2, num_classes = 7)
    yI   = to_categorical(df["IsItalic"], num_classes = 2)
    yB   = to_categorical(df["IsBold"], num_classes = 2)
    return [yFam,yS,yI,yB]

In [ ]:
# Normalize the data
IMG_SIZE = 32

X_train = X_train.reshape(-1,IMG_SIZE,IMG_SIZE,1) / 255.0
X_val = X_val.reshape(-1,IMG_SIZE,IMG_SIZE,1) / 255.0
X_test = X_test.reshape(-1,IMG_SIZE,IMG_SIZE,1) / 255.0

In [ ]:
y_train = dfToY(train)
y_val = dfToY(validate)
y_test = dfToY(test)

In [ ]:
for img in X_train[:10]:
    display(Image.fromarray((img*255).astype(np.uint8).reshape(-1,IMG_SIZE)))

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers as L
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, Input
from tensorflow.keras.optimizers import RMSprop,Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau


In [ ]:
#IMG_SIZE = 32
input = Input(shape=(IMG_SIZE,IMG_SIZE,1))
# x = LSTM(SEQ_LEN)(lstm_input)
x = Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu')(input)
x = Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu')(x)
x = MaxPool2D(pool_size=(2,2))(x)
x = Dropout(0.25)(x)
x = Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu')(x)
x = Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Dropout(0.25)(x)
x = Flatten()(x)

xF = Dense(256, activation = "relu")(x)
xF = Dropout(0.5)(xF)
outputF = Dense(10, activation = "softmax",name='family')(xF)

xS = Dense(256, activation = "relu")(x)
xS = Dropout(0.5)(xS)
outputS = Dense(7, activation = "softmax",name='size')(xS)

xI = Dense(64, activation = "relu")(x)
xI = Dropout(0.5)(xI)
outputI = Dense(2, activation = "softmax",name='italic')(xI)

xB = Dense(64, activation = "relu")(x)
xB = Dropout(0.5)(xB)
outputB = Dense(2, activation = "softmax",name='bold')(xB)


model = Model(inputs=input, outputs=[outputF,outputS,outputI,outputB] )


optimizer = Adam(learning_rate=0.001, decay=1e-6) #RMSprop(learning_rate=0.001, rho=0.9, epsilon=1e-08, decay=0.0)#
model.compile(optimizer = optimizer , loss = "categorical_crossentropy",loss_weights=[1.0, 0.5, 0.25, 0.25], metrics=["accuracy"])

model.summary()
tf.keras.utils.plot_model(model,  show_shapes=True)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [ ]:
epochs = 8 #300
mcp = ModelCheckpoint('save4F.hdf5', save_best_only=True, monitor='val_family_accuracy', mode='max')
es = EarlyStopping(monitor='val_family_accuracy', mode='max', patience=20,restore_best_weights = True)

print("Trainsize",X_train.shape)
history = model.fit(X_train, y_train, epochs = epochs, validation_data = (X_val, y_val), callbacks=[es,mcp],verbose=2) #verbose = 2)

In [ ]:
#break
model.load_weights('../input/train705970-and-weight/Model4Feture_F985_S967_I996_B999.h5')

In [ ]:
dnnTrainLoss = model.evaluate(X_train, y_train)
dnnValLoss = model.evaluate(X_val, y_val)
dnnTestLoss = model.evaluate(X_test, y_test)

In [ ]:
import time
from scipy import stats

def PredictImage(id,image):
    #------------------
    #  ******************************
    #  Your Process Here
    FontFam = 0
    FontSize = 16
    IsItalic = True
    IsBold = True

    imgList = preprocessImage(image,maximg=10)

    if len(imgList) == 0:
        imgList = [image]

    X_final = np.array(imgList).reshape(-1,IMG_SIZE,IMG_SIZE,1)/255.0

    results = model.predict(X_final)
    
    result = np.sum(results[0], axis=0)
    FontFam = np.argmax(result,axis = 0)

    result = np.sum(results[1], axis=0)
    FontSize = (np.argmax(result,axis = 0))*2+12

    result = np.sum(results[2], axis=0)
    IsItalic = np.argmax(result,axis = 0)

    result = np.sum(results[3], axis=0)
    IsBold = np.argmax(result,axis = 0)
    
    return [id,FontFam, FontSize,IsItalic, IsBold]

def PredictImageList(id,X_final):
    #------------------
    #  ******************************
    #  Your Process Here
    FontFam = 0
    FontSize = 16
    IsItalic = True
    IsBold = True

    results = model.predict(X_final)
    
    result = np.sum(results[0], axis=0)
    FontFam = np.argmax(result,axis = 0)

    result = np.sum(results[1], axis=0)
    FontSize = (np.argmax(result,axis = 0))*2+12

    result = np.sum(results[2], axis=0)
    IsItalic = np.argmax(result,axis = 0)

    result = np.sum(results[3], axis=0)
    IsBold = np.argmax(result,axis = 0)
    
    return [id,FontFam, FontSize,IsItalic, IsBold]

In [ ]:
class LoadFormat(TypedDict):
    id : str
    x : float
    y : float
    width : float
    height : float


In [ ]:
from tqdm import tqdm
setpath=datapath +'test' 
testResultT = []
for mainpath in tqdm(os.listdir(setpath)):
    mat = cv2.imread(f"{setpath}/{mainpath}/image.png")
    with open(f"{setpath}/{mainpath}/test.json", encoding="utf-8") as f:
        jSArray : List[LoadFormat] = json.load(f)
    for JSON in jSArray:
        Id = JSON["id"]
        x = JSON["x"]
        y = JSON["y"]
        width = JSON["width"]
        height = JSON["height"]
        Mat = mat[round(y):round(y+height),round(x):round(x+width),:]
        testResultT.append(PredictImage(Id,Mat))
    del mat
len(testResultT)

In [ ]:
setpath=datapath +'final_test' 
finalIdX_final = []
for mainpath in tqdm(os.listdir(setpath)):
    mat = cv2.imread(f"{setpath}/{mainpath}/image.png")
    with open(f"{setpath}/{mainpath}/test.json", encoding="utf-8") as f:
        jSArray : List[LoadFormat] = json.load(f)
    for JSON in jSArray:
        Id = JSON["id"]
        x = JSON["x"]
        y = JSON["y"]
        width = JSON["width"]
        height = JSON["height"]
        Mat = mat[round(y):round(y+height),round(x):round(x+width),:].copy()
        
        imgList = preprocessImage(Mat,maximg=10)
        if len(imgList) == 0:
            imgList = [Mat]

        X_final = np.array(imgList).reshape(-1,IMG_SIZE,IMG_SIZE,1)/255.0
        finalIdX_final.append([Id,X_final])
    del mat
#     break
len(finalIdX_final)

In [ ]:
len(testResultf)

In [ ]:
#np.savez('finalIdMat',Id=finalIdMat,ticker=Xy_Ticker)
# np.save('finalIdX_final', finalIdX_final)
#finalIdX_final = np.load('finalIdX_final.npy',allow_pickle=True)

In [ ]:
setpath=datapath +'final_test' 
testResultf = []
for Id,X_final in tqdm(finalIdX_final):
        testResultf.append(PredictImageList(Id,X_final))
#         break
len(testResultf)

In [ ]:
# setpath=datapath +'final_test' 
# testResultf = []
# for mainpath in tqdm(os.listdir(setpath)):
#     mat = cv2.imread(f"{setpath}/{mainpath}/image.png")
#     with open(f"{setpath}/{mainpath}/test.json", encoding="utf-8") as f:
#         jSArray : List[LoadFormat] = json.load(f)
#     for JSON in jSArray:
#         Id = JSON["id"]
#         x = JSON["x"]
#         y = JSON["y"]
#         width = JSON["width"]
#         height = JSON["height"]
#         Mat = mat[round(y):round(y+height),round(x):round(x+width),:]
#         testResultf.append(PredictImage(Id,Mat))
#     del mat
# #     break
# len(testResultf)

In [ ]:
allresult = []
for x in tqdm(testResultT):
    Id,FontFam, FontSize,IsItalic, IsBold = x
    # FontFam = int(FontFam)
    # IsItalic = int(IsItalic)
    # IsBold = int(IsBold)
    allresult.extend([
        [f"{Id}_name", FontMap[FontFam]],
        [f"{Id}_file", FontFile[FontFam]],
        [f"{Id}_size", f"{FontSize}px"],
        [f"{Id}_style-weight", f"{'italic' if IsItalic else 'normal'}-{'bold' if IsBold else 'normal'}"]])

for x in tqdm(testResultf):
    Id,FontFam, FontSize,IsItalic, IsBold = x
    allresult.extend([
        [f"{Id}_name", FontMap[FontFam]],
        [f"{Id}_file", FontFile[FontFam]],
        [f"{Id}_size", f"{FontSize}px"],
        [f"{Id}_style-weight", f"{'italic' if IsItalic else 'normal'}-{'bold' if IsBold else 'normal'}"]])
len(allresult)

In [ ]:
dfResult = pd.DataFrame (allresult,columns=['Id','myPredicted'])
dfResult

In [ ]:
result = pd.read_csv(datapath+"sample_submission.csv", dtype=str)
result

In [ ]:
result = result.merge(dfResult, on='Id', how='left')
result

In [ ]:
result[["Id","myPredicted"]].rename(columns={'myPredicted': 'Predicted'}).to_csv("submission.csv", index=False) 